In [17]:
pip install mip

Note: you may need to restart the kernel to use updated packages.


In [1]:
from scipy import spatial
from mip import Model, xsum, minimize, BINARY, INTEGER
import pandas as pd

In [15]:
# sample2 엑셀 데이터 불러오기

df_sample = pd.read_csv('sample2.csv')

cost=df_sample[['item', 'T', 'C']].drop_duplicates()
demand=df_sample[['item', 'T', 'D']].drop_duplicates()
prepaid=df_sample[['item', 'T', 'P']].drop_duplicates()
cycle =df_sample[['item', 'J', 'T', 'M']].drop_duplicates()

cost1 = cost.dropna(axis=0)
demand1= demand.dropna(axis=0)
prepaid1= prepaid.dropna(axis=0)
cycle1 = cycle.dropna(axis=0)

#df_sample = pd.read_csv('sample.csv')
#cost=df_sample[['item', 'T', 'C']].drop_duplicates()



In [3]:
ijt_set = pd.DataFrame(df_sample[['item', 'J', 'T']].value_counts()).reset_index()[['item', 'J', 'T']]
ijt_set=ijt_set.sort_values(['item']).reset_index(drop=True)
ijt_set


,item,J,T
0,K04031,409,1
1,K04031,426,1
2,K04031,435,1
3,K04031,416,1
4,K04033,409,1
5,K04033,422,1
6,K04033,435,1
7,K04033,404,1
8,K04037,401,2
9,K04037,409,2


In [4]:
ijt_df = pd.DataFrame(df_sample[['item', 'J', 'T']].value_counts()).reset_index()[['item', 'J', 'T']]
ijt_df=ijt_df.sort_values(['item']).reset_index(drop=True)
ijt_set = set()
for index, info in ijt_df.iterrows():
    ijt_set.add((info['item'], info['J'], info['T']))



In [5]:
Data = pd.read_csv('sample2.csv', index_col=0)
# set

#T is list of time stamps
T = list(set(df_sample.T))

#I is list of all the itens
I = list(set(df_sample.item))

#J is list of all the machines
J= list(set(df_sample.J))


cit = {}
pit = {}
dit = {}
mijt = {}



In [6]:
for index,info in cost1.iterrows():
    #print(info)
    cit[info['item'], info['T']]  =  info['C']
    
for index,info in prepaid1.iterrows():
    #print(info)
    pit[info['item'], info['T']]  =  info['P'] 
    
for index,info in demand1.iterrows():
    #print(info)
    dit[info['item'], info['T']]  =  info['D']
    
for index,info in cycle1.iterrows():
    #print(info)
    mijt[info['item'], info['J'], info['T']]  =  info['M']
   
#cost = pd.read_csv('cost.csv', index_col=0)
#demand = pd.read_csv('demand.csv', index_col=0)
#prepaid = pd.read_csv('prepaid.csv', index_col=0)

In [7]:
#adding missing values

#cit - 단가

for i in I :
  for t in T :
    key = (i,t)
    if key not in cit :
      cit[i,t]=0


#pit - 선급품/일반품

for i in I :
  for t in T :
    key = (i,t)
    if key not in pit :
      pit[i,t]=0

#dit - 생산목표량

for i in I :
  for t in T :
    key = (i,t)
    if key not in dit :
      dit[i,t]=0

#mijt - cycle time

for i in I :
  for j in J:
    for t in T:
      key=(i,j,t)
      if key not in mijt:
        mijt[i,j,t]=0
    


In [8]:
# variables
m = Model("Unist_optimization")
xijt = {
    (i, j, t): m.add_var(var_type=INTEGER, name="x_%s,%s,%s" % (i, j, t))
    for i in I for j in J for t in T
}
uit = {
    (i, t): m.add_var(var_type=INTEGER, name="u_%s,%s" % (i, t))
    for i in I for t in T
}



In [9]:
# objective
m.objective = minimize(
    xsum(uit[i, t]*cit[i, t]*pit[i, t] for i in I for t in T)
    )



In [10]:
# constraint
# Constraint 1:-
for i in I:
    for t in T:
        m += (uit[i, t] + xsum(xijt[i, j, t] for j in J) ==  dit[i, t])

# Constraint 2:-
for j in J:
    for t in T:
        m += (xsum(mijt[i, j, t]*xijt[i, j, t] for i in I) <=  600)

# Constraint 3, 4:-
for i in I:
    for j in J:
        for t in T:
            m += (xijt[i, j, t] >= 0)

for i in I:
    for t in T:
        m += (uit[i, t] >= 0)



In [11]:
# Constraint 5
all_set = set()
for i in I:
    for j in J:
        for t in T:
            if (i, j, t) not in ijt_set:
                m += (xijt[i, j, t] == 0)

In [12]:
m.write('model.lp')
m.read('model.lp')
# %%
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [13]:
solution = []
for i in I:
    for j in J:
        for t in T:
            solution.append([xijt[i, j, t].name, xijt[i, j, t].x])
            solution.append([uit[i, t].name, uit[i, t].x])

solution = pd.DataFrame(
    solution,
    columns=['variable', 'solution'])
solution.sort_values(['variable', 'solution'], inplace=True)
solution.to_csv('solution.csv', index=False)
# %%
m.objective_value
# %%

1377698.0

In [14]:
# solution.csv 파일로 저장됨

solution = pd.read_csv('solution.csv')
solution[solution['solution']>0]

,variable,solution
1,"u_K04031,1",1.0
49,"u_K04033,1",26.0
1372,"x_K04031,416,1",219.0
1564,"x_K04031,426,1",96.0
1660,"x_K04031,435,1",67.0
1804,"x_K04033,404,1",46.0
1900,"x_K04033,409,1",58.0
2092,"x_K04033,422,1",150.0
2332,"x_K04033,435,1",38.0
2439,"x_K04037,401,2",20.0
